# [NTDS'19] assignment 1: network science
[ntds'19]: https://github.com/mdeff/ntds_2019

[Eda Bayram](https://lts4.epfl.ch/bayram), [EPFL LTS4](https://lts4.epfl.ch) and
[Nikolaos Karalias](https://people.epfl.ch/nikolaos.karalias), [EPFL LTS2](https://lts2.epfl.ch).

## Students

* Team: 4 (Yann Yanis Bouquet, Ali El Abridi, Tariq Kalim, Jonas Müller)
* Students: Jonas Müller

## Rules

Grading:
* The first deadline is for individual submissions. The second deadline is for the team submission.
* All team members will receive the same grade based on the team solution submitted on the second deadline.
* As a fallback, a team can ask for individual grading. In that case, solutions submitted on the first deadline are graded.
* Collaboration between team members is encouraged. No collaboration between teams is allowed.

Submission:
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
  Note that Networkx is imported in the second section and cannot be used in the first.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart Kernel and Run All Cells" in Jupyter.

## Objective

The purpose of this milestone is to explore a given dataset, represent it by network by constructing different graphs. In the first section, you will analyze the network properties. In the second section, you will explore various network models and find out the network model fitting the ones you construct from the dataset.

## Cora Dataset

The [Cora dataset](https://linqs.soe.ucsc.edu/node/236) consists of scientific publications classified into one of seven research fields. 

* **Citation graph:** the citation network can be constructed from the connections given in the `cora.cites` file.
* **Feature graph:** each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary and its research field, given in the `cora.content` file. The dictionary consists of 1433 unique words. A feature graph can be constructed using the Euclidean distance between the feature vector of the publications.

The [`README`](data/cora/README) provides details about the content of [`cora.cites`](data/cora/cora.cites) and [`cora.content`](data/cora/cora.content).

## Section 1: Network Properties

In [ ]:
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt

# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

### Question 1: Construct a Citation Graph and a Feature Graph

Read the `cora.content` file into a Pandas DataFrame by setting a header for the column names. Check the `README` file.

In [ ]:
# Your code here.
N_WORDS = 1433
word_n_list = [f'w_{x}' for x in list(range(N_WORDS))]
column_list = ["paper_id"] + word_n_list + ["class_label"]

pd_content = pd.read_csv('data/cora/cora.content', delimiter='\t', names=column_list) 
pd_content.head()
nr_of_papers = len(pd_content)

Print out the number of papers contained in each of the reasearch fields.

**Hint:** You can use the `value_counts()` function.

In [ ]:
pd_content["class_label"].value_counts()

Select all papers from a field of your choice and store their feature vectors into a NumPy array.
Check its shape.

In [ ]:
my_field = "Theory"
my_field_mask = pd_content["class_label"] == my_field

features = pd_content.loc[my_field_mask].drop(["paper_id","class_label"], axis=1).to_numpy()
features.shape

Let $D$ be the Euclidean distance matrix whose $(i,j)$ entry corresponds to the Euclidean distance between feature vectors $i$ and $j$.
Using the feature vectors of the papers from the field which you have selected, construct $D$ as a Numpy array. 

In [ ]:
distance = features[:, :, None] - features[:, :, None].T
distance = np.sqrt((distance**2).sum(1))
distance.shape

Check the mean pairwise distance $\mathbb{E}[D]$.

In [ ]:
mean_distance = distance.mean()
mean_distance

Plot an histogram of the euclidean distances.

In [ ]:
plt.figure(1, figsize=(8, 4))
plt.title("Histogram of Euclidean distances between papers")
plt.hist(distance.flatten());

Now create an adjacency matrix for the papers by thresholding the Euclidean distance matrix.
The resulting (unweighted) adjacency matrix should have entries
$$ A_{ij} = \begin{cases} 1, \; \text{if} \; d(i,j)< \mathbb{E}[D], \; i \neq j, \\ 0, \; \text{otherwise.} \end{cases} $$

First, let us choose the mean distance as the threshold.

In [ ]:
threshold = mean_distance
A_feature = (distance < threshold).astype(int) 
A_feature = A_feature - np.diag(np.ones(A_feature.shape[0]))

np.count_nonzero(A_feature)

Now read the `cora.cites` file and construct the citation graph by converting the given citation connections into an adjacency matrix.

In [ ]:
cora_cites = np.genfromtxt('data/cora/cora.cites', delimiter='\t')

def replace_with_dict(ar, dic):
    # Extract out keys and values
    k = np.array(list(dic.keys()))
    v = np.array(list(dic.values()))

    # Get argsort indices
    sidx = k.argsort()

    # Drop the magic bomb with searchsorted to get the corresponding
    # places for a in keys (using sorter since a is not necessarily sorted).
    # Then trace it back to original order with indexing into sidx
    # Finally index into values for desired output.
    return v[sidx[np.searchsorted(k,ar,sorter=sidx)]]


unique_ids = np.sort(pd.unique(pd_content["paper_id"]))
nr_of_ids = len(unique_ids)

# create dictionary for new indices
dictionary = pd_content["paper_id"].to_dict()
inv_dict = {v: k for k, v in dictionary.items()}

# replace indices
edges = replace_with_dict(cora_cites,inv_dict)
edges = np.array(edges)

# create adjacency matrix
adjacency = np.zeros((nr_of_papers, nr_of_papers), dtype=int)

dataset = pd.DataFrame({'id1': edges[:, 0], 'id2': edges[:, 1]})
for idx, row in dataset.iterrows():
    i, j = int(row.id1), int(row.id2)
    adjacency[i, j] = 1  # weight
    adjacency[j, i] = 1  # weight to obtain an undirected network
    
#adjacency[:15, :15]

A_citation = adjacency
A_citation.shape

In [ ]:
A_citation.shape

In [ ]:
np.unique(A_citation)

Get the adjacency matrix of the citation graph for the field that you chose.
You have to appropriately reduce the adjacency matrix of the citation graph.

In [ ]:
'''
Warning Adjacenecy matrix is A_citation, NOT A_citation_feat
for all the _features stuff we use A_feature
'''
my_field_columns = np.where(np.asarray([my_field_mask]))[1].tolist()

A_citation = A_citation[:,my_field_columns]
A_citation = A_citation[my_field_columns,:]
A_citation.shape

Check if your adjacency matrix is symmetric. Symmetrize your final adjacency matrix if it's not already symmetric.

In [ ]:
np.count_nonzero(A_citation - A_citation.transpose())

Check the shape of your adjacency matrix again.

In [ ]:
A_citation.shape

### Question 2: Degree Distribution and Moments

What is the total number of edges in each graph?

In [ ]:
num_edges_feature = np.count_nonzero(A_feature)/2 
num_edges_citation = np.count_nonzero(A_citation)/2 
print(f"Number of edges in the feature graph: {num_edges_feature}")
print(f"Number of edges in the citation graph: {num_edges_citation}")

Plot the degree distribution histogram for each of the graphs.

In [ ]:
degrees_citation = np.sum(A_citation, axis=0)
degrees_feature = np.sum(A_feature, axis=0)

deg_hist_normalization = np.ones(degrees_citation.shape[0]) / degrees_citation.shape[0]
deg_hist_normalization_feat = np.ones(degrees_feature.shape[0]) / degrees_feature.shape[0]

fig, axes = plt.subplots(1, 2, figsize=(16, 4));
_=axes[0].set_title('Citation graph degree distribution')
_=axes[0].hist(degrees_citation, weights=deg_hist_normalization);
_=axes[0].set_yscale('log')
_=axes[1].set_title('Feature graph degree distribution')
_=axes[1].hist(degrees_feature, weights=deg_hist_normalization_feat);
_=axes[1].set_yscale('log')

Calculate the first and second moments of the degree distribution of each graph.

In [ ]:
cit_moment_1 = np.mean(degrees_citation)
cit_moment_2 = np.mean(degrees_citation**2)

feat_moment_1 = np.mean(degrees_feature)
feat_moment_2 = np.mean(degrees_feature**2)

cit_std = np.std(degrees_citation)
feat_std = np.std(degrees_citation)

print(f"1st moment of citation graph: \t{cit_moment_1:.2f}")
print(f"2nd moment of citation graph: \t{cit_moment_2:.2f}")
print(f"std of citation graph: \t\t{cit_std:.2f}\n")

print(f"1st moment of feature graph: \t{feat_moment_1:.2f}")
print(f"2nd moment of feature graph: \t{feat_moment_2:.2f}")
print(f"std of citation graph: \t\t{feat_std:.2f}")

What information do the moments provide you about the graphs?
Explain the differences in moments between graphs by comparing their degree distributions.

**Your answer here:**  
Calculating the 1st and the 2nd moment from the respective degree distributions gives us the average degree and the standard deviation. 
For all citations we can see that there exists a very small probability for having a high degree node (hub). This increases the first and even more significantly the second moment leading to a standard deviation bigger than the mean degree and thus indicating behaviour of a scale free network.  
For the selected feature graph the histogram shows that the degrees have a lower spread (lower first and second moment) which is anticipated since it is a subset of the citation graph.

Select the 20 largest hubs for each of the graphs and remove them. Observe the sparsity pattern of the adjacency matrices of the citation and feature graphs before and after such a reduction.

In [ ]:
small_node_ind = (degrees_citation.argsort()[:-20])
reduced_A_citation = A_citation[:,small_node_ind]
reduced_A_citation = reduced_A_citation[small_node_ind,:]

small_node_ind = (degrees_feature.argsort()[:-20])
reduced_A_feature = A_feature[:,small_node_ind]
reduced_A_feature = reduced_A_feature[small_node_ind,:]

small_node_ind = np.sort(degrees_citation.argsort()[-20:])
largestHubs = A_citation[:,small_node_ind]
largestHubs = largestHubs[small_node_ind,:]

fig, axes = plt.subplots(2, 2, figsize=(16, 16))
_=axes[0, 0].set_title('Feature graph: adjacency matrix sparsity pattern')
_=axes[0, 0].spy(A_feature);
_=axes[0, 0].set_xlim([0,len(A_feature)])
_=axes[0, 1].set_title('Feature graph without top 20 hubs: adjacency matrix sparsity pattern')
_=axes[0, 1].spy(reduced_A_feature);
_=axes[1, 0].set_title('Citation graph: adjacency matrix sparsity pattern')
_=axes[1, 0].spy(A_citation);
_=axes[1, 1].set_title('Citation graph without top 20 hubs: adjacency matrix sparsity pattern')
_=axes[1, 1].spy(reduced_A_citation);
largestHubs.shape

Plot the new degree distribution histograms.

In [ ]:
reduced_degrees_feat = np.sum(reduced_A_feature, axis=0)# Your code here.
reduced_degrees_cit = np.sum(reduced_A_citation, axis=0)# Your code here.

deg_hist_normalization = np.ones(reduced_degrees_cit.shape[0])/reduced_degrees_cit.shape[0]
deg_hist_normalization_feat = np.ones(reduced_degrees_feat.shape[0]) / reduced_degrees_feat.shape[0]

fig, axes = plt.subplots(1, 2, figsize=(16, 4))
_=axes[0].set_title('Citation graph degree distribution')
_=axes[0].hist(reduced_degrees_cit, weights=deg_hist_normalization);
_=axes[0].set_yscale('log')
_=axes[1].set_title('Feature graph degree distribution')
_=axes[1].hist(reduced_degrees_feat, weights=deg_hist_normalization_feat);
_=axes[1].set_yscale('log')

Compute the first and second moments for the new graphs.

In [ ]:
reduced_cit_moment_1 = np.mean(reduced_degrees_cit)
reduced_cit_moment_2 = np.mean(reduced_degrees_cit**2)

reduced_feat_moment_1 = np.mean(reduced_degrees_feat)
reduced_feat_moment_2 = np.mean(reduced_degrees_feat**2)

reduced_cit_std = np.std(reduced_degrees_cit)
reduced_feat_std = np.std(reduced_degrees_feat)

print(f"Citation graph first moment: \t{reduced_cit_moment_1:.2f}")
print(f"Citation graph second moment: \t{reduced_cit_moment_2:.2f}")
print(f"std of citation graph: \t\t{reduced_cit_std:.2f}\n")

print(f"Feature graph first moment: \t{reduced_feat_moment_1:.2f}")
print(f"Feature graph second moment: \t{reduced_feat_moment_2:.2f}")
print(f"std of feature graph: \t\t{reduced_feat_std:.2f}")

Print the number of edges in the reduced graphs.

In [ ]:
# Your code here
num_edges_reduced_feature = reduced_degrees_feat.sum()/2
num_edges_reduced_citation = reduced_degrees_cit.sum()/2
print(f"Number of edges in the feature graph: \t{num_edges_reduced_feature}")
print(f"Number of edges in the citation graph: \t{num_edges_reduced_citation}\n")

In [ ]:
#print relative change
print(f"Relative change: Citation graph first moment: \t{(reduced_cit_moment_1/cit_moment_1-1)*100:.0f} %")
print(f"Relative change: Citation graph second moment: \t{(reduced_cit_moment_2/cit_moment_2-1)*100:.0f} %")
print(f"Relative change: std of citation graph: \t{(reduced_cit_std/cit_std-1)*100:.0f} %\n")

print(f"Relative change: Feature graph first moment: \t{(reduced_feat_moment_1/feat_moment_1-1)*100:.0f} %")
print(f"Relative change: Feature graph second moment: \t{(reduced_feat_moment_2/feat_moment_2-1)*100:.0f} %")
print(f"Relative change: std of feature graph: \t\t{(reduced_feat_std/feat_std-1)*100:.0f} %\n")

print(f"Relative change: Number of edges in the feature graph:  \t{(num_edges_reduced_feature/num_edges_feature-1)*100:.0f} %")
print(f"Relative change: Number of edges in the citation graph: \t{(num_edges_reduced_citation/num_edges_citation-1)*100:.0f} %")

Is the effect of removing the hubs the same for both networks? Look at the percentage changes for each moment. Which of the moments is affected the most and in which graph? Explain why.  

**Hint:** Examine the degree distributions.

In [ ]:
degrees_citation_sorted = np.sort(degrees_citation)[::-1]
degrees_feature_sorted = np.sort(degrees_feature)[::-1]
n = 30
degrees_citation_sorted[:n]
degrees_feature_sorted[:n]

n = 100
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
_=axes[0].set_title('Citation graph degree distribution')
_=axes[0].plot(degrees_citation_sorted[:n]);
_=axes[1].set_title('Feature graph degree distribution')
_=axes[1].plot(degrees_feature_sorted[:n]);

**Your answer here:**
### may need changes due to update A_feature
Removing the 20 largest hubs significantly affects the second order. This behaviour is reasonable since, due to its definition, it depends on the square of the values and is thus strongly influenced by large values. The first order reduces too, however, considerably less. Furthermore the change is bigger for the feature graph which can be attributet to the fact that we removed the 20 biggest hubs in both cases, meaning that higher percentage of the big hubs in the feature graph has been removed compared to the citation graph due to the different total size.

### Question 3: Pruning, sparsity, paths

By adjusting the threshold of the euclidean distance matrix, prune the feature graph so that its number of edges is roughly close (within a hundred edges) to the number of edges in the citation graph.

In [ ]:
distance.shape

In [ ]:
threshold_pruned = 3.59
a = 0
i = 0
diff = 100;
while abs(diff) > 10 and i < 1:
    if diff > 0:
        a += .01
    else:
        a -= .01
    i += 1
    
    threshold_pruned = threshold_pruned - a

    A_feature_pruned = (distance < threshold_pruned).astype(int)

    # don't count "self-edge" diagonale
    n = A_feature_pruned.shape[0]
    A_feature_pruned[range(n),range(n)] = 0

    num_edges_feature_pruned = np.count_nonzero(A_feature_pruned)/2
    diff = num_edges_feature_pruned-num_edges_feature
    #diff
    #print(f"{a:.4}")
    
threshold_pruned

print(f"Number of edges in the feature graph: \t\t\t{num_edges_feature}")
print(f"Number of edges in the feature graph after pruning: \t{num_edges_feature_pruned}")
print(f"Number of edges in the citation graph: \t\t\t{num_edges_citation}")

Check your results by comparing the sparsity patterns and total number of edges between the graphs.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].set_title('Citation graph sparsity')
axes[0].spy(A_citation);
axes[1].set_title('Feature graph sparsity')
axes[1].spy(A_feature_pruned);

Let $C_{k}(i,j)$ denote the number of paths of length $k$ from node $i$ to node $j$. 

We define the path matrix $P$, with entries
$ P_{ij} = \displaystyle\sum_{k=0}^{N}C_{k}(i,j). $

Calculate the path matrices for both the citation and the unpruned feature graphs for $N =10$.  

**Hint:** Use [powers of the adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix#Matrix_powers).

In [ ]:
def compute_path_matrix(adjacency_matrix, N=10):
    tmp = np.copy(adjacency_matrix)
    paths_of_length_k = np.zeros(N)
    paths_of_length_k[0] = np.count_nonzero(tmp)
    path_matrix = np.eye(tmp.shape[0]) + tmp # we consider the 0-lenght path (adjecency_matrix ** 0 = identity matrix)
    for n in range(1, N):
        tmp = np.matmul(adjacency_matrix, tmp)
        tmp[np.nonzero(tmp)] = 1 # remove multiple path count for counting higher deg. paths
        #np.fill_diagonal(tmp, 0)
        path_matrix += tmp
        paths_of_length_k[n] = np.count_nonzero(tmp)
    paths_of_length_k = paths_of_length_k / path_matrix.size
    #np.fill_diagonal(path_matrix, 0)
    return path_matrix, paths_of_length_k


In [ ]:
path_matrix_citation, citation_paths_of_length_k = compute_path_matrix(A_citation)
path_matrix_feature, feature_paths_of_length_k = compute_path_matrix(A_feature)

Check the sparsity pattern for both of path matrices.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 9))
axes[0].set_title('Citation Path matrix sparsity')
axes[0].spy(path_matrix_citation);
axes[1].set_title('Feature Path matrix sparsity')
axes[1].spy(path_matrix_feature);

Now calculate the path matrix of the pruned feature graph for $N=10$. Plot the corresponding sparsity pattern. Is there any difference?

In [ ]:
path_matrix_pruned, feature_pruned_paths_of_length_k = compute_path_matrix(A_feature_pruned)

plt.figure(figsize=(12, 6))
plt.title('Feature pruned Path matrix sparsity')
plt.spy(path_matrix_pruned);

In [ ]:
np.count_nonzero(path_matrix_citation)
len(path_matrix_citation.flatten())

np.count_nonzero(path_matrix_feature)
len(path_matrix_feature.flatten())

fig, axes = plt.subplots(1, 3, figsize=(18, 4))
_=axes[0].set_title('Citation graph nr of paths (normalized) vs pathlength')
_=axes[0].plot(citation_paths_of_length_k);
_=axes[1].set_title('Feature graph nr of paths (normalized) vs pathlength')
_=axes[1].plot(feature_paths_of_length_k);
_=axes[2].set_title('Feature graph nr of paths (normalized) vs pathlength')
_=axes[2].plot(feature_pruned_paths_of_length_k);

**Your answer here:** The path matrix of the feature and the pruned feature graphs do show some discernible difference. In fact, the pruned feature graphs appears to have more connections between the nodes than the feature graph since the sparsity plot shows more entries. However, this result might be wrong since in the pruned feature graph the biggest hubs have been removed the contrary would be anticipated.

Describe how you can use the above process of counting paths to determine whether a graph is connected or not. Is the original (unpruned) feature graph connected?

**Your answer here:** The path matrix shows how many paths of a length shorter or equal to a chosen length each node has. Therefore, if the number of entries in the path matrix converges to the number of nodes for path matrices with increased path length, this essentially means that at this path length all nodes are connected with at most this path length. As can be seen in the plot above the feature graph is connected with paths of maximum length of two.

If the graph is connected, how can you guess its diameter using the path matrix?

**Your answer here:** By iteratively increasing the exponent of the path matrix and checking if all entries are bigger than zero (connected with less than the path length given by the exponent) the diameter can be found.

If any of your graphs is connected, calculate the diameter using that process.

In [ ]:
old = -1
for n, elem in enumerate(feature_paths_of_length_k):
    if elem == old:
        diameter = n
        break
    else:
        old = elem    
    
print(f"The diameter is: {diameter}")

Check if your guess was correct using [NetworkX](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.distance_measures.diameter.html).
Note: usage of NetworkX is only allowed in this part of Section 1.

In [ ]:
import networkx as nx
feature_graph = nx.from_numpy_matrix(A_feature)
print(f"Diameter according to networkx: {nx.diameter(feature_graph)}")

## Section 2: Network Models

In this section, you will analyze the feature and citation graphs you constructed in the previous section in terms of the network model types.
For this purpose, you can use the NetworkX libary imported below.

In [ ]:
import networkx as nx

Let us create NetworkX graph objects from the adjacency matrices computed in the previous section.

In [ ]:
G_citation = nx.from_numpy_matrix(A_citation)
print('Number of nodes: {}, Number of edges: {}'. format(G_citation.number_of_nodes(), G_citation.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_citation.number_of_selfloops(), nx.number_connected_components(G_citation)))

In the rest of this assignment, we will consider the pruned feature graph as the feature network.

In [ ]:
G_feature = nx.from_numpy_matrix(A_feature_pruned)
print('Number of nodes: {}, Number of edges: {}'. format(G_feature.number_of_nodes(), G_feature.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_feature.number_of_selfloops(), nx.number_connected_components(G_feature)))

### Question 4: Simulation with Erdős–Rényi and Barabási–Albert models

Create an Erdős–Rényi and a Barabási–Albert graph using NetworkX to simulate the citation graph and the feature graph you have. When choosing parameters for the networks, take into account the number of vertices and edges of the original networks.

The number of nodes should exactly match the number of nodes in the original citation and feature graphs.

In [ ]:
assert len(G_citation.nodes()) == len(G_feature.nodes())
n = len(G_citation.nodes())
n

The number of match shall fit the average of the number of edges in the citation and the feature graph.

In [ ]:
m = np.round((G_citation.size() + G_feature.size()) / 2)
m

How do you determine the probability parameter for the Erdős–Rényi graph?

**Your answer here:** The probability p, for the Erdős–Rényi model of any graph, is given by the number of links $m$ of this graph divided by the maximal number of links (unidirectional) possible for the available nodes $n*(n-1)/2$.

In [ ]:
p = (2*m)/(n*(n-1))
G_er = nx.erdos_renyi_graph(n, p)

Check the number of edges in the Erdős–Rényi graph.

In [ ]:
print('My Erdos-Rényi network that simulates the citation graph has {} edges.'.format(G_er.size()))

How do you determine the preferential attachment parameter for Barabási–Albert graphs?

**Your answer here:** Preferential attachemnt of a given graph is the tendency (probability) that newly added nodes link to already highly linked nodes (high degree). This probability $\Pi\left(k_{i}\right)$ that a link of the new node connects to node i can thus be given by $\Pi\left(k_{i}\right)=\frac{k_{i}}{\sum_{j} k_{j}}$

In [ ]:
q = int(m/n)
G_ba = nx.barabasi_albert_graph(n, q)

Check the number of edges in the Barabási–Albert graph.

In [ ]:
print('My Barabási-Albert network that simulates the citation graph has {} edges.'.format(G_ba.size()))

### Question 5: Giant Component

Check the size of the largest connected component in the citation and feature graphs.

In [ ]:
giant_citation = giant_citation = max(list(nx.connected_component_subgraphs(G_citation)), key = len)
print('The giant component of the citation graph has {} nodes and {} edges.'.format(giant_citation.number_of_nodes(), giant_citation.size()))

In [ ]:
giant_feature =  max(list(nx.connected_component_subgraphs(G_feature)), key = len)
print('The giant component of the feature graph has {} nodes and {} edges.'.format(giant_feature.number_of_nodes(), giant_feature.size()))

Check the size of the giant components in the generated Erdős–Rényi graph.

In [ ]:
giant_er = max(list(nx.connected_component_subgraphs(G_er)), key = len)
print('The giant component of the Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er.number_of_nodes(), giant_er.size()))

Let us match the number of nodes in the giant component of the feature graph by simulating a new Erdős–Rényi network.
How do you choose the probability parameter this time? 

**Hint:** Recall the expected giant component size from the lectures.

**Your answer here:**

In [ ]:
ng = giant_feature.number_of_nodes()
p_new = (-np.log(1-(ng/n)))/((ng/n)*(n))
G_er_new = nx.erdos_renyi_graph(n, p_new)

Check the size of the new Erdős–Rényi network and its giant component.

In [ ]:
print('My new Erdos Renyi network that simulates the citation graph has {} edges.'.format(G_er_new.size()))
giant_er_new = max(list(nx.connected_component_subgraphs(G_er_new)), key = len)
print('The giant component of the new Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er_new.number_of_nodes(), giant_er_new.size()))

### Question 6: Degree Distributions

Recall the degree distribution of the citation and the feature graph.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].set_title('Citation graph')
citation_degrees = [G_citation.degree(n) for n in G_citation.nodes()]
axes[0].hist(citation_degrees, bins = range(min(citation_degrees), max(citation_degrees)));
axes[1].set_title('Feature graph')
feature_degrees = [G_feature.degree(n) for n in G_feature.nodes()]
axes[1].hist(feature_degrees, bins = range(min(citation_degrees), max(citation_degrees)));

What does the degree distribution tell us about a network? Can you make a prediction on the network model type of the citation and the feature graph by looking at their degree distributions?

**Your answer here:** By observing both histograms we can see that the number of nodes decreases sharply as the degree increases. It doesn't seem to fit the bell curve that can form a binomial or poisson distribution. Furthermore, by plotting the logarithmic graphs, we can have a better observation of the degree distribution. Let $p_{k}$ be the frequency of the $k$ degree, we can actually see for the citation graph that $log(p_k) \sim - \alpha k $. According to the course, we shall consider that the model that best fit this graph is the scale-free model despite the fact that the distribution doesn't match : it should be $log(p_k) \sim -\gamma log(k)$
Also we cannot make any conclusion about the degree distribution for the feature graph. However 
definit la power law
//We observe that the degree distribution is mostly zero for both graphs. 
(idée : citation graph, nouveau bouquin cite ancien + hubs = crédibilité)
In the citation graph we mostly observe that nodes have zero degree distribution and 
The scale-free behavior of the citation graph could come from the fact that the books are publihed one at a time and considering that here every book are in the same category, the new ones would cite the older ones (growth property) and the more cited books get a scientific credibility that forces the new publication to refer to them intead of the others (preferential attachment propertiy)

Now, plot the degree distribution historgrams for the simulated networks.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].set_title('Citation graph')
citation_degrees = [G_citation.degree(n) for n in G_citation.nodes()]
values, counts = np.unique(citation_degrees,return_counts=True)
counts = counts / len(feature_degrees)
axes[0].plot(values,counts,'o');
axes[1].set_title('Feature graph')
feature_degrees = [G_feature.degree(n) for n in G_feature.nodes()]
values, counts = np.unique(feature_degrees,return_counts=True)
counts = counts / len(feature_degrees)
axes[1].plot(values,counts,'o');

#axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlabel("k")
axes[0].set_ylabel("$p_k$")
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel("k")
axes[1].set_ylabel("$p_k$")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
axes[0].set_title('Erdos-Rényi network')
er_degrees = [G_er.degree(n) for n in G_er.nodes()]
axes[0].hist(er_degrees);
axes[1].set_title('Barabási-Albert network')
ba_degrees = [G_ba.degree(n) for n in G_ba.nodes()]
axes[1].hist(ba_degrees);
axes[2].set_title('new Erdos-Rényi network')
er_new_degrees = [G_er_new.degree(n) for n in G_er_new.nodes()]
axes[2].hist(er_new_degrees);

In terms of the degree distribution, is there a good match between the citation and feature graphs and the simulated networks?
For the citation graph, choose one of the simulated networks above that match its degree distribution best. Indicate your preference below.

**Your answer here:** 

You can also simulate a network using the configuration model to match its degree disctribution exactly. Refer to [Configuration model](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html#networkx.generators.degree_seq.configuration_model).

Let us create another network to match the degree distribution of the feature graph. 

In [ ]:
G_config = nx.configuration_model(feature_degrees, create_using = nx.Graph) 
print('Configuration model has {} nodes and {} edges.'.format(G_config.number_of_nodes(), G_config.size()))

Does it mean that we create the same graph with the feature graph by the configuration model? If not, how do you understand that they are not the same?

**Your answer here:** (TO DOUBLE CHECK) Even though the new graph with configuration_mode that follow the same degree distribution of the feature graph, it does not mean that they may represent the same structure of the graph as the clustering structure may change or the number of connected components even though they have the same degree distribution. 


### Question 7: Clustering Coefficient

Let us check the average clustering coefficient of the original citation and feature graphs. 

In [ ]:
nx.average_clustering(G_citation)

In [ ]:
nx.average_clustering(G_feature)

What does the clustering coefficient tell us about a network? Comment on the values you obtain for the citation and feature graph.

**Your answer here:**

Now, let us check the average clustering coefficient for the simulated networks.

In [ ]:
nx.average_clustering(G_er)

In [ ]:
nx.average_clustering(G_ba)

In [ ]:
nx.average_clustering(nx.Graph(G_config))

Comment on the values you obtain for the simulated networks. Is there any good match to the citation or feature graph in terms of clustering coefficient?

**Your answer here:**

Check the other [network model generators](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html) provided by NetworkX. Which one do you predict to have a better match to the citation graph or the feature graph in terms of degree distribution and clustering coefficient at the same time? Justify your answer.

**Your answer here:**

If you find a better fit, create a graph object below for that network model. Print the number of edges and the average clustering coefficient. Plot the histogram of the degree distribution.

In [ ]:
g_ci_clust = nx.powerlaw_cluster_graph(int(n/2), 2, (np.log(int(n/2)) ** 2)/ (int(n/2) ** 2) )

In [ ]:
g_ci_clust.size()

Comment on the similarities of your match.

**Your answer here:**

In [ ]:
nx.average_clustering(g_ci_clust)